<div class="alert alert-block alert-info" style="margin-top: 20px">

      
| Name | Description | Date
| :- |-------------: | :-:
|Reza Hashemi| Multi Layer Perceptrons - 1st  | Finalized on 23rd of August 2019 | width="750" align="center"></a></p>
</div>


# Multi Layer Perceptrons
- Breast cancer prediction with MLP
  - [Breast cancer dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer)

In [0]:
!pip3 install torch torchvision

In [0]:
import numpy as np
import pandas as pd
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
torch.__version__

'1.1.0'

## 1. Import & process dataset


In [0]:
from sklearn.datasets import load_breast_cancer
# define dataset
class breastCancerDataset(torch.utils.data.Dataset):
  def __init__(self):
    data = load_breast_cancer()

    self.X = data.data
    self.y = data.target
  
  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]
  
  def __len__(self):
    return len(self.X)

In [0]:
# create dataset instance and train-test split
dataset = breastCancerDataset()

NUM_INSTANCES = len(dataset)
TEST_RATIO = 0.4
TEST_SIZE = int(NUM_INSTANCES * TEST_RATIO)
TRAIN_SIZE = NUM_INSTANCES - TEST_SIZE

print(TRAIN_SIZE, TEST_SIZE)

342 227


In [0]:
train_data, test_data = torch.utils.data.random_split(dataset, (TRAIN_SIZE, TEST_SIZE))

train_loader = torch.utils.data.DataLoader(train_data, batch_size = 128, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 128, shuffle = False)

## 2. Creating MLP model and training

- Create and train MLP with 1 hidden layer

![](http://cs231n.github.io/assets/nn1/neural_net.jpeg)

In [0]:
# create MLP with one hidden layer
class net(nn.Module):
  def __init__(self, input_dim, hidden_dim):
    super(net, self).__init__()
    self.dense1 = nn.Linear(input_dim, hidden_dim)   # hidden layer
    self.relu = nn.ReLU()
    self.dense2 = nn.Linear(hidden_dim, 2)           # output layer
  
  def forward(self, x):
    x = self.dense1(x)   
    x = self.relu(x)
    x = self.dense2(x)
    return x

In [0]:
# hyperparameters
DEVICE = torch.device('cuda')
INPUT_DIM = load_breast_cancer().data.shape[-1]
HIDDEN_DIM = 5
LEARNING_RATE = 1e-3
NUM_EPOCHS = 10

In [0]:
model = net(INPUT_DIM, HIDDEN_DIM).to(DEVICE)
criterion = nn.CrossEntropyLoss()   # do not need softmax layer when using CEloss criterion
optimizer = torch.optim.RMSprop(model.parameters(), lr = LEARNING_RATE)

In [0]:
# training for NUM_EPOCHS
for i in range(NUM_EPOCHS):
  temp_loss = []
  for (x, y) in train_loader:
    x, y = x.float().to(DEVICE), y.to(DEVICE)
    outputs = model(x)
    loss = criterion(outputs, y)
    temp_loss.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  print("Loss at {}th epoch: {}".format(i, np.mean(temp_loss)))

Loss at 0th epoch: 17.05094846089681
Loss at 1th epoch: 0.31504593789577484
Loss at 2th epoch: 0.31400464475154877
Loss at 3th epoch: 0.35018259286880493
Loss at 4th epoch: 0.28126850227514905
Loss at 5th epoch: 0.39963358640670776
Loss at 6th epoch: 0.27110442022482556
Loss at 7th epoch: 0.23624453445275626
Loss at 8th epoch: 0.29986998438835144
Loss at 9th epoch: 0.45256727933883667


## 3. Evaluation
- Evaluate the trained MLP model with ROC AUC score 
  - Store probability of each instance to a list and compare it with true y label

In [0]:
# record probability and true label 
y_prob, y_true = [], []
with torch.no_grad():
  for x, y in test_loader:
    x, y = x.float().to(DEVICE), y.to(DEVICE)
    outputs = F.softmax(model(x))
    y_true += list(y.cpu().numpy())                # true label
    y_prob += list(outputs.cpu().numpy()[:, -1])   # predicted probability 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


In [0]:
# evaluation result
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_prob)

0.9275325553864366